In [1]:
import pandas as pd

In [35]:
# data1 = pd.read_csv("properties_1.csv")
data2 = pd.read_csv("properties_2.csv")
data3 = pd.read_csv("properties_3.csv")

In [36]:
data = pd.concat([data2,data3])

In [37]:
data = data.drop(columns=["Unnamed: 0"])

In [38]:
data = data.drop_duplicates()

In [128]:
data

,name,location,rent,deposit,area,type,highlights,description,owner
0,Majestique Towers,"2 BHK Flat for rent in Kharadi, Pune","₹30,000",NaN,808 sqft,2 BHK,"['2 Baths', 'East Facing']",A 2 bhk flat for rent in kharadi close to all ...,PW Realty
1,Balaji Mesmero,"2 BHK Flat for rent in Lohegaon, Pune","₹22,500",+ Deposit 2 months rent,"1,040 sqft",2 BHK,"['2 Baths', 'East Facing']",2 bhk flat with modular kitchen lift garden ki...,NK Properties
2,ARV Ganga Kingston,"2 BHK Flat for rent in NIBM, Pune","₹27,000",+ Deposit 3 months rent,"1,012 sqft",2 BHK,"['2 Baths', 'Private Garden']",Rent: 22000,Foursides Property Management
3,Goel Ganga Constella,"2 BHK Flat for rent in Eon Free Zone , Pune","₹37,000",+ Deposit 3 months rent,960 sqft,2 BHK,"['East Facing', 'Full Power Backup']",2 bhk flat (East facing) with covered car park...,Owner
4,Solitaire Residency,"3 BHK Flat for rent in Rahatani, Pune","₹35,000","+ Deposit ₹90,000","1,444 sqft",3 BHK,"['2 Baths', 'Full Power Backup']",The rose valley society is the one of the famo...,Bardhaman Property one of the famous consultan...
...,...,...,...,...,...,...,...,...,...
190,Tain Square,"3 BHK Flat for rent in Fatima Nagar, Pune","₹35,000",+ Deposit 3 months rent,"1,550 sqft",3 BHK,"['3 Baths', 'East Facing']",Tain square is the most preferred destination ...,VISIION REAL ESTATE
191,Ganga Melrose,"2 BHK Flat for rent in Sopan Baug, Pune","₹39,000",+ Deposit 3 months rent,"1,400 sqft",2 BHK,"['2 Baths', 'East Facing']",Ganga melrose is one of pune's most sought aft...,VISIION REAL ESTATE
192,ARV Ganga Kingston,"2 BHK Flat for rent in NIBM, Pune","₹21,000","+ Deposit ₹60,000","1,020 sqft",2 BHK,"['2 Baths', 'Full Power Backup']",This lovely 2 bhk apartment for rent in nibm i...,VISIION REAL ESTATE
193,Puraniks Aldea,"2 BHK Flat for rent in Baner, Pune","₹28,000","+ Deposit ₹70,000","1,080 sqft",2 BHK,"['2 Baths', 'Gated Society']",This 2 bhk flat in baner is available in puran...,shree ganesh real estate


In [163]:
df = data.copy()
df = df.reset_index()
df = df.drop(columns="index")

In [164]:
def strip_location(loc):
    res = loc.split("in")[1].replace(" ","").split(",")[0]
    return res

In [165]:
df['location'] = df['location'].apply(strip_location)
df["location"] = df["location"].replace("H", "Hadapsar")

In [166]:
def strip_rent(rent):
    if 'Lac' in rent:
        res = float(rent.split(" ")[0].replace("₹", "")) * 100000
    else:
        res = float(rent.replace("₹", "").replace(",", ""))
    return res

In [167]:
df['rent'] = df['rent'].apply(strip_rent)

In [168]:
def strip_area(area):
    res = area.split(" ")[0]
    res = res.replace(",","")
    return float(res)

In [169]:
df['area'] = df['area'].apply(strip_area)

In [170]:
import re
def extract_deposit_amount(rent,deposit):
    direct_amount_match = re.search(r"\₹([\d,]+)", deposit)
    if direct_amount_match:
        return float(
            direct_amount_match.group(1).replace(",", "")
        )
    months_match = re.search(r"(\d+)\s+months\s+rent", deposit)
    if months_match:
        num_months = int(months_match.group(1))
        return num_months * rent
    return None

In [171]:
for index, row in df.iterrows():
    try:
        if pd.notna(row["deposit"]):  # Check if 'deposit' is not NaN
            new_deposit = extract_deposit_amount(row["rent"], row["deposit"])
            df.at[index, "deposit"] = (
                new_deposit  # Update 'newdeposit' column in the DataFrame
            )
    except Exception as e:
        df.at[index, "deposit"] = None  # Set 'deposit' to None if an exception occurs

In [172]:
df

,name,location,rent,deposit,area,type,highlights,description,owner
0,Majestique Towers,Kharadi,30000.0,NaN,808.0,2 BHK,"['2 Baths', 'East Facing']",A 2 bhk flat for rent in kharadi close to all ...,PW Realty
1,Balaji Mesmero,Lohegaon,22500.0,45000.0,1040.0,2 BHK,"['2 Baths', 'East Facing']",2 bhk flat with modular kitchen lift garden ki...,NK Properties
2,ARV Ganga Kingston,NIBM,27000.0,81000.0,1012.0,2 BHK,"['2 Baths', 'Private Garden']",Rent: 22000,Foursides Property Management
3,Goel Ganga Constella,EonFreeZone,37000.0,111000.0,960.0,2 BHK,"['East Facing', 'Full Power Backup']",2 bhk flat (East facing) with covered car park...,Owner
4,Solitaire Residency,Rahatani,35000.0,90000.0,1444.0,3 BHK,"['2 Baths', 'Full Power Backup']",The rose valley society is the one of the famo...,Bardhaman Property one of the famous consultan...
...,...,...,...,...,...,...,...,...,...
611,Tain Square,FatimaNagar,35000.0,105000.0,1550.0,3 BHK,"['3 Baths', 'East Facing']",Tain square is the most preferred destination ...,VISIION REAL ESTATE
612,Ganga Melrose,SopanBaug,39000.0,117000.0,1400.0,2 BHK,"['2 Baths', 'East Facing']",Ganga melrose is one of pune's most sought aft...,VISIION REAL ESTATE
613,ARV Ganga Kingston,NIBM,21000.0,60000.0,1020.0,2 BHK,"['2 Baths', 'Full Power Backup']",This lovely 2 bhk apartment for rent in nibm i...,VISIION REAL ESTATE
614,Puraniks Aldea,Baner,28000.0,70000.0,1080.0,2 BHK,"['2 Baths', 'Gated Society']",This 2 bhk flat in baner is available in puran...,shree ganesh real estate
